### Mounting drive on Colab

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


###Libraries

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [3]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Number of Neurons per hidden layer ratio to input layer (1/3, 1/2, 2/3, 3/4, 1)

### For Imagenet as a source

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50

from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()


pre_trained_model = ResNet50(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)
#for layer in pre_trained_model.layers:
#  layer.trainable = False

In [6]:
def compile_model(rate):
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(int(2000*rate), activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(int(2000 * (rate) * (rate)), activation='relu')(x)                  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [7]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [8]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [9]:
for i in [1/3, 1/2, 2/3, 3/4, 1]:
  print("******************* Training model for ratio of", i, "************************")
  imagenet_resnet50 = compile_model(i)
  history = imagenet_resnet50.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
  print("******************* Model evaluation for a ratio of", i, "*************************")
  eval = imagenet_resnet50.evaluate(X_test, y_test)
  print("******************* Train time is", sum(cb.logs), "***********")
  print("f1 score is", sum(eval[6])/15)

******************* Training model for ratio of 0.3333333333333333 ************************


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 83s 296ms/step - loss: 3.1888 - acc: 0.0830 - auc: 0.5392 - recall: 0.0053 - precision: 0.0939 - hamming_loss: 0.9947 - f1_score: 0.0097 - val_loss: 2.7474 - val_acc: 0.0797 - val_auc: 0.5287 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 60s 302ms/step - loss: 2.2474 - acc: 0.2330 - auc: 0.7763 - recall: 0.0422 - precision: 0.5727 - hamming_loss: 0.9578 - f1_score: 0.0719 - val_loss: 2.5567 - val_acc: 0.1957 - val_auc: 0.7097 - val_recall: 0.0427 - val_precision: 0.2596 - val_hamming_loss: 0.9573 - val_f1_score: 0.0482
Epoch 3/10
200/200 [==============================] - 60s 300ms/step - loss: 1.5373 - acc: 0.4523 - auc: 0.9102 - recall: 0.2563 - precision: 0.6824 - hamming_loss: 0.7437 - f1_score: 0.3427 - val_loss: 2.6422 - val_acc: 0.2963 - val_auc: 0.7856 - val_recall: 0.1997 - val_precision: 0.3702 - val_hamming_l